In [77]:
import pandas as pd
import numpy as np
from dateutil.parser import parse
import time
from datetime import datetime

In [78]:
data = pd.read_csv('../data/helpdesk.csv')
#data['CompleteTimestamp'] = data['CompleteTimestamp'].apply(parse)

In [79]:
data.head()

,CaseID,ActivityID,CompleteTimestamp
0,2,1,2012-04-03 16:55:38
1,2,8,2012-04-03 16:55:53
2,2,6,2012-04-05 17:15:52
3,3,1,2010-10-29 18:14:06
4,3,8,2010-11-04 01:16:11


In [103]:
df = data[0:5]

In [104]:
df

,CaseID,ActivityID,CompleteTimestamp
0,2,1,2012-04-03 16:55:38
1,2,8,2012-04-03 16:55:53
2,2,6,2012-04-05 17:15:52
3,3,1,2010-10-29 18:14:06
4,3,8,2010-11-04 01:16:11


In [105]:
for i in range(0, len(df)-1):
    starting = parse(df.loc[i,'CompleteTimestamp'])
    ending = parse(df.loc[i+1,'CompleteTimestamp'])
    df.loc[i+1,'Duration'] = (ending-starting).total_seconds()

/Users/hoangnguyen/miniconda3/envs/pydata/lib/python3.5/site-packages/pandas/core/indexing.py:297: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/hoangnguyen/miniconda3/envs/pydata/lib/python3.5/site-packages/pandas/core/indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [107]:
df.fillna(0)

,CaseID,ActivityID,CompleteTimestamp,Duration
0,2,1,2012-04-03 16:55:38,0.0
1,2,8,2012-04-03 16:55:53,15.0
2,2,6,2012-04-05 17:15:52,173999.0
3,3,1,2010-10-29 18:14:06,-45270106.0
4,3,8,2010-11-04 01:16:11,457325.0


In [61]:
def calculateDuration(df):
    df['Duration'] = (df['CompleteTimestamp'] - df['CompleteTimestamp'].shift(1)).fillna(0)
    return df

def calculateCumDuration(df):
    df['CumDuration'] = (df['CompleteTimestamp'] - df['CompleteTimestamp'].iloc[0]).fillna(0) 
    #change df['CompleteTimestamp'][0] --> df['CompleteTimestamp'].iloc[0]
    return df

def calculateTimeSinceMidNight(x):
    x = str(x)
    x = time.strptime(x, "%Y-%m-%d %H:%M:%S")
    midnight = datetime.fromtimestamp(time.mktime(x)).replace(hour=0, minute=0, second=0, microsecond=0)
    timesincemidnight = datetime.fromtimestamp(time.mktime(x))-midnight
    return timesincemidnight.seconds

def convert2seconds(x):
    x = int(x.total_seconds())
    return x

In [ ]:
def compute_daily_return(df):
    #return (df/df.shift(1)) - 1
    daily_returns = df.copy()
    daily_returns[1:] = (df[1:]/ df[:-1].values) - 1
    daily_returns.ix[0, :] = 0
    return daily_returns


def compute_cumulative_return(df):
    daily_returns = df.copy()
    daily_returns[1:] = (df[1:]/ df[:-1].values) - 1 #need to modify here
    daily_returns.ix[0, :] = 0
    return (df/df[0]) - 1